In [1]:
import numpy as np
import csv
import time

np.random.seed(1234)
def randomize(): np.random.seed(time.time())

In [2]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

In [3]:
# 실험용 메인 함수
def abalone_exec(epoch_count=10, mb_size=10, report=1):
    load_abalone_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [4]:
# 데이터 적재 함수 정의
def load_abalone_dataset():
    with open('../../data/chap01/abalone.data.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None) # 첫 행 스킵 (헤더)
        rows = []
        for row in csvreader:
            rows.append(row)
    
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1 # 퍼셉트론 입력, 출력의 크기 지정
    data = np.zeros([len(rows), input_cnt + output_cnt])
    
    for n, row in enumerate(rows):
        # 원핫 벡터 변환 처리 
        if row[0] == 'I': data[n, 0] = 1
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        # 성별 이외의 정보를 일괄 복제 
        data[n, 3:] = row[1:]

In [5]:
# 파라미터 초기화 함수 정의
# 퍼셉트론의 가중치 파라미터 weight와 bias를 초기화 
def init_model():
    # 전역 변수로 선언하여 다른 함수에서도 사용 가능 
    # input_cnt, output_cnt는 load_abalone_dataset 함수에서 전역으로 선언되어 있어 바로 사용 가능 
    global weight, bias, input_cnt, output_cnt
    # 가중치 행렬 값을 정규 분포를 갖는 랜덤값으로 초기화 
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    # 초기에 지나친 영향을 끼치지 않도록 0으로 초기화 
    bias = np.zeros([output_cnt])

In [6]:
# 학습 및 평가 함수 정의
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch + 1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'.\
                 format(epoch+1, np.mean(losses), np.mean(accs), acc))
    
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))
    

In [7]:
# 학습 및 평가 데이터 획득 함수 정의
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0]) # 데이터의 수만큼 일련번호 발생
    np.random.shuffle(shuffle_map) # 무작위로 순서 섞음
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global adta, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size * nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]
    

In [ ]:
# 학습 실행함수와 평가 실행 함수 정의
def run_train(x, y):
    # 순전파 처리 
    output, aux_nn = forward_neuralnet(x) # 단층 퍼셉트론 신경망에 대한 순전파 수행 => 입력 행렬 x로부터 신경망 출력 output 계산
    loss, aux_pp = forward_postproc(output, y) # output과 y 로부터 손실함 수 loss 계산
    # 신경망 계산 => 후처리 의 두가지 단계로 나누면 독립적으로 다른 형태로 교체하기 편리 
    # aux_nn, aux_pp 값은 역전파에 사용되는 정보 
    accuracy = eval_accuracy(output, y) # 정확도를 출력
    
    # 역전파 처리 
    # 역전파 함수는 항상 순전파의 역순으로 실행되어야 함
    # 순전파 때 출력이었던 성분의 손실 기울기를 내부 처리하여 순전파 때 입력이었던 성분의 손실 기울기를 반환해야 하기 때문
    G_loss = 1.0 # 역전파의 시작점 
    G_ouput = backprop_postproc(G_loss, aux_pp) # G_loss로부터 G_output을 구한다.
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

    